# Install the dotnet agent

## Build

1. Clone the agent from repo.
2. Add it to the dotnet solution and project (SabberStonePython).
3. Build the solution.
4. Run

```shell
cd dotnet_core/ai
git clone https://github.com/rnilva/Hearthstone-MonteCarloGraphSearch

cd ../
dotnet sln add ai/Hearthstone-MonteCarloGraphSearch/MonteCarloGraphSearch/MonteCarloGraphSearch.csproj
dotnet add SabberStonePython.csproj reference ai/Hearthstone-MonteCarloGraphSearch/MonteCarloGraphSearch/MonteCarloGraphSearch.csproj

dotnet build -c Release -o ../python/_sabberstone_dotnet

dotnet ../python/_sabberstone_dotnet/SabberStonePython.dll
```

# Python API

## Structure of AbstractAI

```python
class AbstractAI(ABC):
    def __init__(self, stub, account_name):
        self._stub = stub
        self.metadata = (('id', account_name),)
        self.game_id = ""
        self.response_stream = None
        self.request_stream = None

    @abstractmethod
    def get_option(self, game: Game, sabber_stub) -> Option:
        """Calculate and reutnr the best move
        with regard to the given game state.
        Each AI agent must implement this method.
        """
        pass

    @abstractmethod
    def on_match_started(self):
        """This method will be called when a match is started."""
        pass

    @abstractmethod
    def on_game_started(self):
        """This method will be called when each game(round) is started."""
        pass

    @abstractmethod
    def on_game_finished(self, game):
        """This method will be called when each game(round) is finished."""
        pass
```


## Run SabberStone gRPC stub

In [ ]:
import sys
sys.path.insert(0, "../../../")

In [ ]:
import grpc
from pysabberstone.python.rpc import python_pb2_grpc

In [ ]:
channel = grpc.insecure_channel("localhost:50052")
stub = python_pb2_grpc.SabberStonePythonStub(channel)
dotnet_ai_stub = python_pb2_grpc.DotnetAIServiceStub(channel)

## Create Python `AbstractAI` instance
In this tutorial we are going to use `RandomAI`.

In [ ]:
from pysabberstone.python.ai.abstract_ai import RandomAI, dotnet_ai_match

In [ ]:
account_name = "TestPythonRandomAI"
ai = RandomAI(stub, account_name)

## Run `dotnet_ai_match()`

In [ ]:
deck = r"AAEBAQcCrwSRvAIOHLACkQP/A44FqAXUBaQG7gbnB+8HgrACiLACub8CAA=="

dotnet_ai_match(python_ai=ai,
                dotnet_ai_name="MonteCarloGraphSearch",
                python_ai_deckstring=deck,
                dotnet_ai_deckstring=deck,
                sabber_stub=stub,
                dotnet_ai_stub=dotnet_ai_stub,
                history=True)

## Implementation of `dotnet_ai_match()`

```python
def dotnet_ai_match(python_ai: AbstractAI, dotnet_ai_name: str,
                    python_ai_deckstring, dotnet_ai_deckstring,
                    sabber_stub: SabberStonePythonStub,
                    dotnet_ai_stub: DotnetAIServiceStub,
                    history: bool = False,
                    random_seed: int = 0,
                    count: int = 1):
    """Run matches between python AI agent and dotnet AI agent."""
    python_ai.on_match_started()
    for i in range(count):
        python_ai.on_game_started()
        # Request: Initialise dotnet AI agent with a specified name and
        #          get new established game.
        #          If dotnet AI is the first player of the game, it will
        #          play its turn.
        #          The game used in matches is partially observable, i.e.,
        #          the opponent's hand and deck is filled with 'Unknown' cards.
        response = dotnet_ai_stub.Request(
            DotnetAIRequest(dotnet_ai_name=dotnet_ai_name,
                            dotnet_ai_deckstring=dotnet_ai_deckstring,
                            python_ai_deckstring=python_ai_deckstring,
                            history=history,
                            seed=random_seed))
        if response.Type == 1:  # NOT_FOUND
            print("dotnet agent with name {} is not found.".format(
                dotnet_ai_name))
            return
        elif response.Type == 2:  # OCCUPIED
            print("Only one match can be run.")
            return
        elif response.Type == 3:  # INVALID_DECKSTRING
            print("Invalid deckstring.")
            return

        game = response.game

        # Main loop
        while game.state != 3:
            # Get option from the python ai
            option = python_ai.get_option(game, sabber_stub)

            # Send Option: If the given option is end turn option,
            #              python client receives the start of the next
            #              turn after the dotnet client plays its turn.
            #              Therefore 'game' is always python client's turn.
            game = dotnet_ai_stub.SendPythonAIOption(option)

        # TODO: winner

        python_ai.on_game_finished(game)
```

# Watch replays with GUI

## History output from SabberStonePython

1. To save replay log, you should call `dotnet_ai_match()` with `history=True` option.
2. Saved replay logs are located `{DOTNET_DLL_DIRECTORY}/history` .

## SabberStoneUnityClient

1. Get SabberStoneUnityClient, build and run.
2. Click 'replay' button.
3. Load a saved history log.
4. Click 'NEXT STEP' button on the lower left corner.
5. Enjoy!